## Amazon AgentCore Bedrock Code Interpreter を使用した高度なデータ分析 - チュートリアル (LangChain)
このチュートリアルでは、Python を使用してコード実行により高度なデータ分析を行う AI エージェントの作成方法を説明します。LLM によって生成されたコードを実行するために Amazon Bedrock AgentCore Code Interpreter を使用します。

このチュートリアルでは、AgentCore Bedrock Code Interpreter を使用して以下を行う方法を説明します：
1. サンドボックス環境のセットアップ
2. ユーザークエリに基づいてコードを生成し高度なデータ分析を行う LangChain ベースのエージェントの設定
3. Code Interpreter を使用したサンドボックス環境でのコード実行
4. ユーザーへの結果表示

## 前提条件
- Bedrock AgentCore Code Interpreter へのアクセス権を持つ AWS アカウント
- Code Interpreter リソースを作成・管理するために必要な IAM 権限
- 必要な Python パッケージのインストール（boto3、bedrock-agentcore、LangChain を含む）
- IAM ロールには Amazon Bedrock でモデルを呼び出す権限が必要
 - US Oregon (us-west-2) リージョンの Claude 3.7 Sonnet モデルへのアクセス

## IAM 実行ロールには以下の IAM ポリシーをアタッチする必要があります

~~~ {
"Version": "2012-10-17",
"Statement": [
    {
        "Effect": "Allow",
        "Action": [
            "bedrock-agentcore:CreateCodeInterpreter",
            "bedrock-agentcore:StartCodeInterpreterSession",
            "bedrock-agentcore:InvokeCodeInterpreter",
            "bedrock-agentcore:StopCodeInterpreterSession",
            "bedrock-agentcore:DeleteCodeInterpreter",
            "bedrock-agentcore:ListCodeInterpreters",
            "bedrock-agentcore:GetCodeInterpreter"
        ],
        "Resource": "*"
    },
    {
        "Effect": "Allow",
        "Action": [
            "logs:CreateLogGroup",
            "logs:CreateLogStream",
            "logs:PutLogEvents"
        ],
        "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
    }
]
}

## 動作の仕組み

コード実行サンドボックスは、Code Interpreter、シェル、ファイルシステムを備えた隔離環境を作成することで、エージェントがユーザーのクエリを安全に処理できるようにします。Large Language Model がツール選択を支援した後、このセッション内でコードが実行され、その後ユーザーまたはエージェントに返されて統合されます。

![architecture local](code-interpreter.png)

## 1. 環境のセットアップ

まず、必要なライブラリをインポートし、Code Interpreter クライアントを初期化します。

デフォルトのセッションタイムアウトは 900 秒（15 分）です。ただし、データの詳細な分析を行うため、少し長いセッションタイムアウト時間 1200 秒（20 分）でセッションを開始します。

In [ ]:
!pip install --upgrade -r requirements.txt

In [ ]:
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
from langchain.agents import AgentExecutor, create_tool_calling_agent, initialize_agent, tool
from langchain_aws import ChatBedrockConverse
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
import json
import pandas as pd
from typing import Dict, Any, List

# Initialize the Code Interpreter within a supported AWS region.
code_client = CodeInterpreter('us-west-2')
code_client.start(session_timeout_seconds=1200)

## 2. ローカルデータファイルの読み込み

サンプルデータファイルの内容を読み込みます。このファイルは Name、Preferred_City、Preferred_Animal、Preferred_Thing の 4 列と約 300,000 件のレコードを含むランダムデータで構成されています。

後ほどエージェントを使用してこのファイルを分析し、分布と外れ値を理解します。

In [ ]:
df_data = pd.read_csv("samples/data.csv")
df_data.head()

In [ ]:
def read_file(file_path: str) -> str:
    """エラーハンドリング付きでファイル内容を読み込むヘルパー関数"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"エラー: ファイル '{file_path}' が見つかりませんでした。")
        return ""
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return ""

data_file_content = read_file("samples/data.csv")

## 3. サンドボックス環境用ファイルの準備

サンドボックス環境に作成するファイルを定義する構造を作成します。

In [ ]:
files_to_create = [
                {
                    "path": "data.csv",
                    "text": data_file_content
                }]

## 4. ツール呼び出し用ヘルパー関数の作成

このヘルパー関数は、サンドボックスツールの呼び出しとレスポンスの処理を容易にします。アクティブなセッション内では、サポートされている言語（Python、JavaScript）でコードを実行し、依存関係の設定に基づいてライブラリにアクセスし、可視化を生成し、実行間で状態を維持できます。

In [ ]:
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    """Helper function to invoke sandbox tools

    Args:
        tool_name (str): Name of the tool to invoke
        arguments (Dict[str, Any]): Arguments to pass to the tool

    Returns:
        Dict[str, Any]: JSON formatted result
    """
    response = code_client.invoke(tool_name, arguments)
    for event in response["stream"]:
        return json.dumps(event["result"])

## 5. Code Sandbox へのデータファイル書き込み

データファイルをサンドボックス環境に書き込み、正常に作成されたことを確認します。

In [ ]:
# Write files to sandbox
writing_files = call_tool("writeFiles", {"content": files_to_create})
print("ファイル書き込み結果:")
print(writing_files)

# Verify files were created
listing_files = call_tool("listFiles", {"path": ""})
print("\nサンドボックス内のファイル:")
print(listing_files)

## 6. LangChain ベースのエージェントを使用した高度な分析の実行

ここで、上記でサンドボックスにアップロードしたデータファイルに対してデータ分析を行うエージェントを設定します。

### 6.1 システムプロンプトの定義
AI アシスタントの動作と機能を定義します。回答は常にコード実行とデータに基づく推論によって検証するようアシスタントに指示します。

In [ ]:
SYSTEM_PROMPT = """あなたは提供されたツールを使用してコード実行によりすべての回答を検証する親切な AI アシスタントです。ツールを使用せずに質問に回答しないでください。

検証の原則：
1. コード、アルゴリズム、計算について主張する場合は、それを検証するコードを書く
2. execute_python を使用して数学的計算、アルゴリズム、ロジックをテストする
3. 回答を出す前に理解を検証するテストスクリプトを作成する
4. 常に実際のコード実行で作業を示す
5. 不確かな場合は、制限を明示し、可能な範囲で検証する

アプローチ：
- プログラミングの概念について質問された場合、コードで実装して実演する
- 計算を求められた場合、プログラムで計算しコードを示す
- アルゴリズムを実装する場合、正確性を証明するテストケースを含める
- 透明性のために検証プロセスを文書化する
- サンドボックスは実行間で状態を維持するため、以前の結果を参照できる

利用可能なツール：
- execute_python: Python コードを実行して出力を確認

レスポンス形式：execute_python ツールは以下を含む JSON レスポンスを返します：
- sessionId: サンドボックスセッション ID
- id: リクエスト ID
- isError: エラーの有無を示すブール値
- content: type と text/data を含むコンテンツオブジェクトの配列
- structuredContent: コード実行の場合、stdout、stderr、exitCode、executionTime を含む

コード実行が成功した場合、出力は content[0].text および structuredContent.stdout に含まれます。
エラーがあったかどうかを確認するには isError フィールドをチェックしてください。

徹底的で正確であり、可能な限り常に回答を検証してください。"""

### 6.2 コード実行ツールの定義
次に、エージェントがコードサンドボックスでコードを実行するためのツールとして使用する関数を定義します。@tool デコレータを使用して、エージェントのカスタムツールとして関数をアノテーションします。

アクティブな Code Interpreter セッション内では、サポートされている言語（Python、JavaScript）でコードを実行し、依存関係の設定に基づいてライブラリにアクセスし、可視化を生成し、実行間で状態を維持できます。

In [ ]:
# Code Interpreter ツールの定義と設定
@tool
def execute_python(code: str, description: str = "") -> str:
    """サンドボックスで Python コードを実行します。"""

    if description:
        code = f"# {description}\n{code}"

    # 実行される生成コードを出力
    print(f"\n 生成されたコード: {code}")


    # Invoke メソッドを呼び出し、初期化された Code Interpreter セッション内で生成されたコードを実行
    response = code_client.invoke("executeCode", {
        "code": code,
        "language": "python",
        "clearContext": False
    })
    for event in response["stream"]:
        return json.dumps(event["result"])

### 6.3 エージェントの設定
LangChain SDK を使用してエージェントを作成・設定します。上で定義したシステムプロンプトとコード生成実行用ツールを提供します。

#### 6.4 言語モデルの初期化

Claude Haiku 4.5 モデルを使用するために、[Cross-region Inference (CRIS)](https://docs.aws.amazon.com/bedrock/latest/userguide/cross-region-inference.html) プロファイル ID を指定します。

In [ ]:
llm = ChatBedrockConverse(model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",region_name="us-west-2")

#### 6.5 プロンプトテンプレートの定義

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

#### 6.6 カスタムツールのリストを作成

In [ ]:
tools = [execute_python]

### 6.7 エージェントエグゼキュータの作成

In [ ]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## 7. エージェントの呼び出しとレスポンス処理
クエリでエージェントを呼び出し、レスポンスを処理します。


注意：非同期実行には非同期環境での実行が必要です。

## 7.1 探索的データ分析（EDA）を実行するクエリ

まず、コードサンドボックス環境内のデータファイルに対して探索的データ分析を実行するようエージェントに指示するクエリから始めましょう。

In [ ]:
query = "ファイル 'data.csv' を読み込み、探索的データ分析（EDA）を実行してください。分布と外れ値について教えてください。"

response=agent_executor.invoke({"input": query})
print("\n*********最終結果*********")
print(response['output'][0]['text'])

## 7.2 情報抽出のクエリ

次に、コードサンドボックス環境内のデータファイルから特定の情報を抽出するようエージェントに指示します。

In [ ]:
query = "ファイル 'data.csv' 内で、名前が 'Kimberly' で、お気に入りの動物が 'Crocodile' の人は何人いますか？"

response=agent_executor.invoke({"input": query})
print("\n*********最終結果*********")
print(response['output'][0]['text'])

## 8. クリーンアップ

最後に、Code Interpreter セッションを停止してクリーンアップします。セッションの使用が完了したら、リソースを解放し不要な課金を避けるためにセッションを停止する必要があります。

In [ ]:
# Stop the Code Interpreter session
code_client.stop()
print("Code Interpreter セッションが正常に停止しました！")